<a href="https://colab.research.google.com/github/gvnbleid/2020-GUM/blob/master/MGU_trzeci_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobieranie danych z Kaggle

In [0]:
!mkdir .kaggle
!mkdir /root/.kaggle
import json
token = {"username":"jacekmyna","key":"7eff50cbfe2c482b7125064101dc821d"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c tensorflow-speech-recognition-challenge -p /content
!7z x train.7z -o/content/data

Podpięcie dysku Google

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Biblioteki

In [0]:
# path
import os
from os.path import isdir, join
from pathlib import Path

# Scientific Math 
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.offline as py
import plotly.graph_objs as go

# Deep learning
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, LSTM
from keras.layers import Activation, TimeDistributed, Embedding, CuDNNLSTM 
from keras.layers import Bidirectional, Permute, Reshape, Flatten, Lambda
from keras.layers import Conv2D, Dot, Softmax
from keras import Input, layers, utils, optimizers, losses
from keras import backend as K
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.backend import squeeze

from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D

import random
import copy
import librosa

Ładowanie danych i preprocessing

In [0]:
train_audio_path = 'data/train/audio/'

dirs = [f for f in os.listdir(train_audio_path) if isdir(join(train_audio_path, f))]
dirs.sort()
print('Number of labels: ' + str(len(dirs[1:])))
print(dirs)

all_wav = []
unknown_wav = []
label_all = []
label_value = {}
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
unknown_list = [d for d in dirs if d not in target_list and d != '_background_noise_' ]
print('target_list : ',end='')
print(target_list)
print('unknowns_list : ', end='')
print(unknown_list)
print('silence : _background_noise_')
i = 0
background = [f for f in os.listdir(join(train_audio_path, '_background_noise_')) if f.endswith('.wav')]
background_noise = []

Konwersja z użyciem librosa

In [0]:
for wav in background : 
    samples, sample_rate = librosa.load(join(join(train_audio_path,'_background_noise_'),wav))
    samples = librosa.resample(samples, sample_rate, 8000)
    background_noise.append(samples) 

for direct in dirs[1:]:
    waves = [f for f in os.listdir(join(train_audio_path, direct)) if f.endswith('.wav')]
    label_value[direct] = i
    i = i + 1
    print(str(i)+":" +str(direct) + " ")
    
    for wav in waves:
        samples, sample_rate = librosa.load(join(join(train_audio_path,direct),wav), sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        
        if len(samples) != 8000: 
            continue
            
        if direct in unknown_list:
            unknown_wav.append(samples)
        else:
            label_all.append(direct)
            all_wav.append([samples, direct])
           
wav_all = np.reshape(np.delete(all_wav,1,1),(len(all_wav)))
label_all = [i for i in np.delete(all_wav,0,1).tolist()]

# Random pick start point
def get_one_noise(noise_num = 0):
    selected_noise = background_noise[noise_num]
    start_idx = random.randint(0, len(selected_noise)- 1 - 8000)
    return selected_noise[start_idx:(start_idx + 8000)]
  
max_ratio = 0.1
noised_wav = []
augment = 1
delete_index = []
for i in range(augment):
    new_wav = []
    noise = get_one_noise(i)
    for i, s in enumerate(wav_all):
        if len(s) != 8000:
            delete_index.append(i)
            continue
        s = s + (max_ratio * noise)
        noised_wav.append(s)
np.delete(wav_all, delete_index)
np.delete(label_all, delete_index)


wav_vals = np.array([x for x in wav_all])
label_vals = [x for x in label_all]
wav_vals.shape

labels = copy.deepcopy(label_vals)
for _ in range(augment):
    label_vals = np.concatenate((label_vals, labels), axis = 0)
label_vals = label_vals.reshape(-1,1)

# knowns audio random sampling
unknown = unknown_wav
np.random.shuffle(unknown_wav)
unknown = np.array(unknown)
unknown = unknown[:2000*(augment+1)]
unknown_label = np.array(['unknown' for _ in range(2000*(augment+1))])
unknown_label = unknown_label.reshape(2000*(augment+1),1)

delete_index = []
for i,w in enumerate(unknown):
    if len(w) != 8000:
        delete_index.append(i)
unknown = np.delete(unknown, delete_index, axis=0)

# silence audio
silence_wav = []
num_wav = (2000*(augment+1))//len(background_noise)
for i, _ in enumerate(background_noise):
    for _ in range((2000*(augment+1))//len(background_noise)):
        silence_wav.append(get_one_noise(i))
silence_wav = np.array(silence_wav)
silence_label = np.array(['silence' for _ in range(num_wav*len(background_noise))])
silence_label = silence_label.reshape(-1,1)
silence_wav.shape

wav_vals    = np.reshape(wav_vals,    (-1, 8000))
noised_wav  = np.reshape(noised_wav,  (-1, 8000))
unknown       = np.reshape(unknown,   (-1, 8000))
silence_wav = np.reshape(silence_wav, (-1, 8000))

print(wav_vals.shape)
print(noised_wav.shape)
print(unknown.shape)
print(silence_wav.shape)

print(label_vals.shape)
print(unknown_label.shape)
print(silence_label.shape)

Zapis danych do plików numpy

In [0]:
save_path = 'gdrive/My Drive/DL2020/Projekt3/numpy_data'

with open(join(save_path, 'wav_vals.npy'), 'wb+') as f:
    np.save(f, wav_vals)
with open(join(save_path, 'noised_wav.npy'), 'wb+') as f:
    np.save(f, noised_wav)
with open(join(save_path, 'unknown.npy'), 'wb+') as f:
    np.save(f, unknown)
with open(join(save_path, 'silence_wav.npy'), 'wb+') as f:
    np.save(f, silence_wav)
with open(join(save_path, 'label_vals.npy'), 'wb+') as f:
    np.save(f, label_vals)
with open(join(save_path, 'unknown_label.npy'), 'wb+') as f:
    np.save(f, unknown_label)
with open(join(save_path, 'silence_label.npy'), 'wb+') as f:
    np.save(f, silence_label)

Ładowanie danych z plików

In [0]:
load_path = 'gdrive/My Drive/DL2020/Projekt3/numpy_data'

wav_vals = np.load(join(load_path, 'wav_vals.npy'))
noised_wav = np.load(join(load_path, 'noised_wav.npy'))
unknown = np.load(join(load_path, 'unknown.npy'))
silence_wav = np.load(join(load_path, 'silence_wav.npy'))
label_vals = np.load(join(load_path, 'label_vals.npy'))
unknown_label = np.load(join(load_path, 'unknown_label.npy'))
silence_label = np.load(join(load_path, 'silence_label.npy'))

print(wav_vals.shape)
print(noised_wav.shape)
print(unknown.shape)
print(silence_wav.shape)

print(label_vals.shape)
print(unknown_label.shape)
print(silence_label.shape)

Podział na zbiory treningowy i testowy

In [0]:
wav_vals = np.concatenate((wav_vals, noised_wav), axis = 0)
wav_vals = np.concatenate((wav_vals, unknown), axis = 0)
wav_vals = np.concatenate((wav_vals, silence_wav), axis = 0)

label_vals = np.concatenate((label_vals, unknown_label), axis = 0)
label_vals = np.concatenate((label_vals, silence_label), axis = 0)

print(len(wav_vals))
print(len(label_vals))

train_wav, test_wav, train_label, test_label = train_test_split(wav_vals, label_vals, 
                                                                    test_size=0.2,
                                                                    random_state = 1993,
                                                                   shuffle=True)

Parametry uczenia i sieci

In [0]:
lr = 0.001
batch_size = 512
drop_out_rate = 0.5
input_shape = 8000
units = 10
recur_layers = 10
embed_dim = 32
lstm_out = 10

Parametry do zapisu modelu

In [0]:
save_dir = 'gdrive/My Drive/DL2020/Projekt3/saved_models/'
model_name = 'v1_trained_model.h5'

Preprocessingu cd.

In [0]:
label_value = target_list
label_value.append('unknown')
label_value.append('silence')

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

# Make Label data 'string' -> 'class num'
temp = []
for v in train_label:
    temp.append(label_value[v[0]])
train_label = np.array(temp)

temp = []
for v in test_label:
    temp.append(label_value[v[0]])
test_label = np.array(temp)

# Make Label data 'class num' -> 'One hot vector'
train_label = utils.to_categorical(train_label, len(label_value))
test_label = utils.to_categorical(test_label, len(label_value))

print('Train_Wav Demension : ' + str(np.shape(train_wav)))

print('Train_Label Demension : ' + str(np.shape(train_label)))

print('Test_Wav Demension : ' + str(np.shape(test_wav)))

print('Test_Label Demension : ' + str(np.shape(test_label)))

print('Number Of Labels : ' + str(len(label_value)))

Przykładowy model

In [0]:
!pip uninstall tensorflow
!pip install tensorflow==1.13.2
!pip install tensorflow-gpu==1.13.2

In [0]:
num_classes = len(label_value)

def create_model():
    x_in = Input((8000,))

    x = Reshape((1, -1))(x_in)

    x = Melspectrogram(n_dft=1024, n_hop=128, input_shape=(1, 8000),
                       padding='same', sr=8000, n_mels=80,
                       fmin=20.0, fmax=8000 / 2, power_melgram=1.0,
                       return_decibel_melgram=True, trainable_fb=False,
                       trainable_kernel=False,
                       name='mel_stft')(x)
    x = Normalization2D(int_axis=0)(x)
    x = Permute((2, 1, 3))(x)

    x = Conv2D(10, (5, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    # x = Reshape((125, 80)) (x)
    x = Lambda(lambda q: squeeze(q, -1), name='squeeze_last_dim')(x)  # keras.backend.squeeze(x, axis)

    x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(x)  # [b_s, seq_len, vec_dim]
    x = Bidirectional(CuDNNLSTM(32, return_sequences=True))(x)  # [b_s, seq_len, vec_dim]

    x_first = Lambda(lambda q: q[:, 32])(x)  # [b_s, vec_dim]
    query = Dense(64)(x_first)

    # dot product attention
    attention_scores = Dot(axes=[1, 2])([query, x])
    attention_scores = Softmax(name='attSoftmax')(attention_scores)  # [b_s, seq_len]

    # rescale sequence
    attention_vector = Dot(axes=[1, 1])([attention_scores, x])  # [b_s, vec_dim]

    x = Dense(32, activation='relu')(attention_vector)
    x = Dense(16)(x)

    output = Dense(num_classes, activation='softmax', name='output')(x)

    return Model(inputs=x_in, outputs=output)

model = create_model()

Ustawienie checkpointu

In [0]:
# checkpoint
filepath="gdrive/My Drive/DL2020/Projekt3/saved_models/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Trenowanie modelu

In [0]:
import tensorflow as tf
from tensorflow import keras

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [0]:
model.compile(optimizer='sgd', loss='mean_squared_error')
history = model.fit(train_wav, train_label, validation_data=[test_wav, test_label],
          batch_size=batch_size, 
          epochs=100,
          callbacks=callbacks_list,
          verbose=1)

Zapis modelu do pliku

In [0]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Wypakowanie danych testowych

In [0]:
!7z x test.7z -o/content/data

Preprocessing na danych testowych

In [0]:
test_audio_path = 'data/test/audio/'

label_value = {}
target_list = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

label_value = target_list
label_value.append('unknown')
label_value.append('silence')

new_label_value = dict()
for i, l in enumerate(label_value):
    new_label_value[l] = i
label_value = new_label_value

all_wav = []

waves = [f for f in os.listdir(test_audio_path) if f.endswith('.wav')]
for wav in waves:
  samples, sample_rate = librosa.load(join(test_audio_path, wav), sr = 16000)
  samples = librosa.resample(samples, sample_rate, 8000)
  if len(samples) != 8000 : 
    continue
  all_wav.append([samples, wav])
          
wav_all = np.reshape(np.delete(all_wav,1,1),(len(all_wav)))

test_data = np.array([x for x in wav_all])

test_data = test_data.reshape(-1,8000)

print(len(test_data))

Załadowanie modelu

In [0]:
model_path = os.path.join(save_dir, model_name)
model.load_weights(model_path)

Predykcja na zbiorze testowym

In [0]:
y_prob = model.predict(test_data) 
y_classes = y_prob.argmax(axis=-1)

Zapis wyników na dysk

In [0]:
it = 0
with open('gdrive/My Drive/DL2020/Projekt3/results' + str(xStart) + '.csv', mode='w') as output_file:
  output_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for y in y_classes:
    output_writer.writerow([all_fnames[it], target_list[y]])
    it = it + 1